In [39]:
from elasticsearch import Elasticsearch
import beir

import os

from beir import util

from elasticsearch.helpers import streaming_bulk

from tqdm.notebook import tqdm

In [45]:
es.indices.stats()

{'_shards': {'total': 4, 'successful': 2, 'failed': 0},
 '_all': {'primaries': {'docs': {'count': 171332, 'deleted': 31500},
   'store': {'size_in_bytes': 52992723, 'reserved_in_bytes': 0},
   'indexing': {'index_total': 410164,
    'index_time_in_millis': 37939,
    'index_current': 0,
    'index_failed': 0,
    'delete_total': 0,
    'delete_time_in_millis': 0,
    'delete_current': 0,
    'noop_update_total': 0,
    'is_throttled': False,
    'throttle_time_in_millis': 0},
   'get': {'total': 0,
    'time_in_millis': 0,
    'exists_total': 0,
    'exists_time_in_millis': 0,
    'missing_total': 0,
    'missing_time_in_millis': 0,
    'current': 0},
   'search': {'open_contexts': 0,
    'query_total': 5,
    'query_time_in_millis': 248,
    'query_current': 0,
    'fetch_total': 5,
    'fetch_time_in_millis': 92,
    'fetch_current': 0,
    'scroll_total': 0,
    'scroll_time_in_millis': 0,
    'scroll_current': 0,
    'suggest_total': 0,
    'suggest_time_in_millis': 0,
    'suggest

In [15]:
# dataset = "trec-covid"
# url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
# out_dir = os.path.join(os.getcwd(), "datasets")
# data_path = util.download_and_unzip(url, out_dir)

/Users/gregorypolyakov/MIPT_Search_engine/datasets/trec-covid.zip: 100%|██████████| 70.5M/70.5M [00:10<00:00, 7.21MiB/s]


In [16]:
from beir.datasets.data_loader import GenericDataLoader

data_path = "datasets/trec-covid"
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test") 

100%|██████████| 171332/171332 [00:01<00:00, 95131.64it/s] 


In [5]:
es = Elasticsearch('localhost')

language = "english"

In [6]:
mapping = {"mappings" : {
                    "properties" : {
                        "title": {"type": "text", "analyzer": language},
                        "text": {"type": "text", "analyzer": language}
                    }}}

In [28]:
es.indices.delete(index="test_index", ignore=[400, 404]) 

{'acknowledged': True}

In [29]:
es.indices.create(index="test_index", body=mapping, ignore=[400])

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_index'}

In [30]:
actions = []
        
for key in corpus.keys():
    actions.append({
        "_id": str(key),
        "_op_type": "index",
        "refresh": "wait_for",
        "title": corpus[key].get("title", None),
        "text": corpus[key].get("text", None),
    })

In [31]:
for output in tqdm(streaming_bulk(client=es, index="test_index", actions=actions)):
    pass

In [37]:
query = "covid statistics in Russia"

req_body = {"query" : {"multi_match": {
                       "query": "Causes of COVID", 
                       "type": "best_fields",
                       "fields": ["text", "title"],
                       "tie_breaker": 0.5
                       }}}
        

res = es.search(
    search_type="dfs_query_then_fetch",
    index = "test_index", 
    body = req_body, 
    size = 10
)

In [38]:
res

{'took': 57,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 9.96275,
  'hits': [{'_index': 'test_index',
    '_type': '_doc',
    '_id': '9yb9a9vz',
    '_score': 9.96275,
    '_source': {'refresh': 'wait_for',
     'title': 'Causes of Death and Comorbidities in Patients with COVID-19',
     'text': 'Infection by the new corona virus strain SARS-CoV-2 and its related syndrome COVID-19 has caused several hundreds of thousands of deaths worldwide. Patients of higher age and with preexisting chronic health conditions are at an increased risk of fatal disease outcome. However, detailed information on causes of death and the contribution of comorbidities to death yet is missing. Here, we report autopsy findings on causes of death and comorbidities of 26 decedents that had clinically presented with severe COVID-19. We found that septic shock and multi organ failure was the most 